

# Relax interface using EMT potential
Optimize atoms coordinates of the interface using the EMT (Effective Medium Theory) potential and BFGS.

<b style="color:red">NOTE:</b> The [EMT potential](https://wiki.fysik.dtu.dk/ase/ase/calculators/emt.html) is available for a limited number of elements (Al, Cu, Ag, Au, Ni, Pd and Pt, as well as H, C, N, O in a limited way). If the interface contains elements not supported by EMT, the relaxation will not be performed.

<h2 style="color:green">Usage</h2>

1. To get interface material one can run [Create Interface with Min Strain ZSL](create_interface_with_min_strain_zsl.ipynb) notebook first. Upon completion, the resulting interface from it will be accessible in the current notebook as explained in [Data exchange](Introduction.ipynb#data-exchange).
1. Set the relaxation parameter in cell 1.1. below (or use the default value).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Perform relaxation of the interface with set parameters
1. View the structure before and after relaxation

## Notes
1. More detailed relaxation parameters can be set in cell 2.1. 

## 1. Prepare the Environment
### 1.1. Set up the relaxation parameter


In [ ]:
# Maximum force tolerance for the relaxation to stop, in eV/Å
FMAX = 0.05

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb")

### 1.3. Get input interface material
Materials are loaded with `get_data()`.

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
interface = materials[0]

### 1.4. Preview Interface

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize(interface, repetitions=[3, 3, 1], rotation="0x")

## 2. Perform Relaxation
### 2.1. Set the optimization parameters

In [ ]:
from ase.optimize import BFGS
from ase.calculators.emt import EMT
from mat3ra.made.tools.convert import to_ase

calculator = EMT()
optimizer = BFGS
OPTIMIZATION_PARAMETERS = {
    "FMAX": FMAX
}

### 2.2. Optimize atomic coordinates of the interface and view energy/step plot

In [ ]:
from utils.plot import create_realtime_plot, plot_update_callback
from mat3ra.made.tools.convert import from_ase
from mat3ra.made.material import Material

# Add calculator to the interface for relaxation
ase_interface = to_ase(interface)
ase_interface.set_calculator(calculator)

dyn = optimizer(ase_interface)
steps = []
energies = []

fig = create_realtime_plot()

# Run the relaxation
dyn.attach(plot_update_callback(dyn, ase_interface, fig, steps, energies), interval=1)
dyn.run(fmax=OPTIMIZATION_PARAMETERS["FMAX"])

ase_final_interface = ase_interface
final_interface = Material(from_ase(ase_final_interface))

### 2.3. View structure before and after relaxation

In [ ]:
visualize([{"material": interface, "title": "original"}, {"material":final_interface, "title":"relaxed"}], rotation= "-90x", repetitions=[3, 3, 1])


### 2.4. Calculate energy using ASE EMT
The interfacial energy is the sum of the surface energies of the substrate and film minus the adhesion energy. According to Dupré's formula

In [ ]:
from mat3ra.made.tools.calculate import calculate_total_energy, calculate_interfacial_energy

original_energy = calculate_total_energy(interface, calculator)
relaxed_energy = calculate_total_energy(final_interface, calculator)
interfacial_energy = calculate_interfacial_energy(interface=final_interface, calculator=calculator)

# Print out the metrics
print(f"Starting interface energy: {original_energy:.4f} eV")
print(f"Relaxed interface energy: {relaxed_energy:.4f} eV")
print(f"Interfacial energy: {interfacial_energy:.4f} eV")

## 3. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
final_interface.name = f"{interface.name}, Relaxed with EMT" if "Relaxed" not in interface.name else interface.name
set_materials(final_interface)